In [19]:
import librosa
import pydub

In [20]:
import IPython.display
import librosa.display

In [114]:
path = "dataset/first_wav.wav"

In [176]:
sample_freq = 16000   #decide quality
duration = 3

In [132]:
data, sample_freq = librosa.load(path, sr = sample_freq, duration = duration)

In [133]:
data,data.shape

(array([ 0.        ,  0.        ,  0.        , ..., -0.10729638,
        -0.10755295, -0.13323691], dtype=float32), (48000,))

In [93]:
IPython.display.Audio(data = data, rate = sample_freq)

In [134]:
data = librosa.to_mono(data)

### stft

In [119]:
import tensorflow as tf
import functools
from tensorflow.contrib.framework.python.ops import audio_ops
from tensorflow.python.ops import io_ops
from tensorflow.contrib.signal.python.ops import window_ops

In [30]:
"""
tf.contrib.signal.stft(
    signals,
    frame_length,
    frame_step,
    fft_length=None,
    window_fn=functools.partial(window_ops.hann_window, periodic=True),
    pad_end=False,
    name=None
)"""

'\ntf.contrib.signal.stft(\n    signals,\n    frame_length,\n    frame_step,\n    fft_length=None,\n    window_fn=functools.partial(window_ops.hann_window, periodic=True),\n    pad_end=False,\n    name=None\n)'

In [138]:
??librosa.stft

In [148]:
lib_stft = librosa.stft(data, n_fft = 512, hop_length = 160, win_length = 400, center = False)

In [149]:
lib_stft.shape

(257, 297)

In [142]:
data_t = tf.convert_to_tensor(data); data.shape

(48000,)

In [150]:
stft = tf.contrib.signal.stft(data_t, frame_length = 400, frame_step = 160, fft_length = 512,
                             window_fn = functools.partial(window_ops.hann_window, periodic=True))
stft.shape

TensorShape([Dimension(298), Dimension(257)])

In [166]:
?? audio_ops.decode_wav

In [188]:
with tf.Session(graph=tf.Graph()) as sess:
    holder = tf.placeholder(tf.string, [])
    file = tf.read_file(holder)
    decoder = audio_ops.decode_wav(file, desired_channels = 1, desired_samples = sample_freq* duration)
    data = sess.run(decoder, feed_dict = {holder:"first_wav.wav"}) ; print(data.audio.shape)

(48000, 1)


In [190]:
stft = tf.contrib.signal.stft(tf.transpose(data.audio), frame_length = 400, frame_step = 160, fft_length = 512,
                             window_fn = functools.partial(window_ops.hann_window, periodic=True)) ; stft.shape

TensorShape([Dimension(1), Dimension(298), Dimension(257)])

### Power law compression

In [193]:
stft = stft ** 0.3

In [194]:
stft

<tf.Tensor 'pow_1:0' shape=(1, 298, 257) dtype=complex64>

## Scrapy

In [185]:
print((16000 * 25)/ 1000) #-> frame length
print((16000 * 10)/ 1000) # -> hop length -> frame_step

400.0
160.0
